In [1]:
import $file.common
import common._
import doobie.implicits._

import $file.$     

import common._

import doobie.implicits._

# Variation 2. In-memory queries

Let's pretend that we can use in-memory data structures to consult and manipulate the information stored in the world database. In Scala, we would commonly use case classes, `List`s, `Map`s, and other data types from the Scala standard library:

In [2]:
case class Country(
    code: String, 
    name: String, 
    capital: Option[Int])

case class City(
    id: Int, 
    name: String, 
    countryCode: String, 
    population: Int)

case class World(
    countries: Map[String, Country],
    cities: Map[Int, City]){
    
    val allCountries: List[Country] = 
        countries.values.toList
    val allCities: List[City] = 
        cities.values.toList
}

defined class Country
defined class City
defined class World

The `largeCapitals` query can then be written using for-comprehensions (which build upon common higher-order functions `flatMap`, `filter` and `map`) in a very concise and readable way:

In [3]:
def largeCapitals(implicit world: World): List[(String, String)] =
    for {
        Country(_, name, Some(capitalId)) <- world.allCountries
        city <- world.cities.get(capitalId)
        if city.population > 8000000
    } yield (city.name, name)

defined function largeCapitals

### Modularity FTW!

This is just fine, but in a more complex setting, we may greatly benefit from decomposing our queries into smaller and reusable building blocks. This would lead to a query made from query fragments as follows: 

In [4]:
def largeCity(maybeId: Option[Int], size: Long)(implicit world: World): Option[City] = 
    for {
        cityId <- maybeId
        city <- world.cities.get(cityId)
        if size < city.population
    } yield city

def largeCapitalsM(implicit world: World): List[(String, String)] = 
    for {
        country <- world.allCountries
        city <- largeCity(country.capital, 8000000)
    } yield (city.name, country.name)

defined function largeCity
defined function largeCapitalsM

### What about testing?

In a general-purpose language setting, we also strive for unit testing.

In [5]:
import org.scalatest._

class LargeCapitalsSpec(largeCapitals: World => List[(String, String)])
extends FlatSpec with Matchers{
    
    val smallWorld: World =         
        World(Map("ES" -> Country("ES","Spain",Some(0)),
                "USA" -> Country("USA", "United States", Some(1)),
                "UK" -> Country("UK", "United Kingdom", Some(2)),
                "UNK" -> Country("UNK", "Unknown", None)),
        Map(0->City(0,"Madrid","ES",9000000),
            1->City(1,"Washington", "USA", 10000000),
            2->City(2,"London", "UK", 500000)))    
    
    "large capitals" should "be right" in {
        largeCapitals(smallWorld).toSet shouldBe 
            Set(("Madrid", "Spain"), ("Washington", "United States"))
    }
}

import org.scalatest._


defined class LargeCapitalsSpec

In [6]:
run(new LargeCapitalsSpec(largeCapitals(_)))
run(new LargeCapitalsSpec(largeCapitalsM(_)))

cmd4$Helper$LargeCapitalsSpec:
large capitals
- should be right
cmd4$Helper$LargeCapitalsSpec:
large capitals
- should be right


### But this is all too impractical, right?

This is most idiomatic code in Scala, but, of course, normally datasets don't fit into main memory. In our sample world database, it run without problems: 

In [7]:
val realWorld = World(
    Map.from(
        sql"select code, code, name, capital from country"
            .query[(String, Country)].to[List].transact(xa).unsafeRunSync),
    Map.from(
        sql"select id, id, name, countryCode, population from city"
            .query[(Int, City)].to[List].transact(xa).unsafeRunSync))

realWorld: World = World(
  HashMap(
    "CYM" -> Country("CYM", "Cayman Islands", Some(553)),
    "MNG" -> Country("MNG", "Mongolia", Some(2696)),
    "SYR" -> Country("SYR", "Syria", Some(3250)),
    "LTU" -> Country("LTU", "Lithuania", Some(2447)),
    "GMB" -> Country("GMB", "Gambia", Some(904)),
    "UZB" -> Country("UZB", "Uzbekistan", Some(3503)),
    "MAC" -> Country("MAC", "Macao", Some(2454)),
    "KHM" -> Country("KHM", "Cambodia", Some(1800)),
    "ROM" -> Country("ROM", "Romania", Some(3018)),
    "BMU" -> Country("BMU", "Bermuda", Some(191)),
    "GUF" -> Country("GUF", "French Guiana", Some(3014)),
    "SWE" -> Country("SWE", "Sweden", Some(3048)),
    "ARM" -> Country("ARM", "Armenia", Some(126)),
    "LAO" -> Country("LAO", "Laos", Some(2432)),
    "SPM" -> Country("SPM", "Saint Pierre and Miquelon", Some(3067)),
    "GNQ" -> Country("GNQ", "Equatorial Guinea", Some(2972)),
    "USA" -> Country("USA", "United States", Some(3813)),
    "CXR" -> Country("CXR", "Christmas

In [8]:
largeCapitals(realWorld).timed(100).nanos

250892 nanos


res7: List[(String, String)] = List(
  ("Jakarta", "Indonesia"),
  ("Ciudad de M\u00e9xico", "Mexico"),
  ("Moscow", "Russian Federation"),
  ("Seoul", "South Korea")
)

Ideally, we would like to program our queries as we did here, with higher-order functions, and be able to access the database. How do we do that? Let's see our first contender, [DAOs](Variation3.DAOs.ipynb)!